Demonstration notebook for end users

In [ ]:
# Visual imports
from moviepy.video.io.VideoFileClip import VideoFileClip
from IPython.display import display, clear_output
from PIL import Image
from collections import deque
from visualization_utils import draw_trakcs

# Physics import
from mosquito_detector import MosquitoDetector, AdaptiveThreshold
from mosquito_tracker import MosquitoTracker
import torch

In [ ]:
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
clip = VideoFileClip("data/mosquito_videos/1.mp4")
video = clip.iter_frames(fps=30)

In [ ]:
kernel_size = 10

adaptive_threshold = AdaptiveThreshold(rate=0.1, percentile=0.99)
detector = MosquitoDetector(adaptive_threshold, kernel_size=kernel_size, stride=2, sigma=3, device=device)

frame_buffer = deque(maxlen=10)
frame_buffer.append(detector.grayscale(torch.from_numpy(next(video).copy()).to(device)))
frame_buffer.append(detector.grayscale(torch.from_numpy(next(video).copy()).to(device)))

tracker = MosquitoTracker(min_hits=7, max_age=5, threshold_radius=80, frame_size=frame_buffer[0].shape, device=device)

In [ ]:
for i, np_frame in enumerate(video):
    frame = detector.grayscale(torch.from_numpy(np_frame.copy()).to(device))
    frame_buffer.append(frame)

    centers, conf = detector.detect(frame_buffer)

    tracks = tracker.update(centers, conf)
    drawed_frame = draw_trakcs(np_frame, tracks, kernel_size, centers=True)
    clear_output(wait=True)
    display(Image.fromarray(drawed_frame))